In [107]:
import pandas as pd
import streamlit as st
import plotly.express as px
import numpy as np

# File and folder path
fol = ''
fn = 'FS_Golf_DB.xlsx'
df = pd.read_excel(fol+fn)


In [108]:
dimensions = ['Club','Golfer','Session','Shot_Type','Mode']

num_columns = ['Ball_mph', 'Club_mph', 'Smash_Factor', 'Carry_yds','Total_yds', 'Roll_yds', 'Swing_H', 'Height_ft', 'Time_s', 'AOA',
       'Spin_Loft', 'Swing_V', 'Spin_Axis', 'Lateral_yds', 'FTP', 'FTT','Dynamic_Loft', 'Club_Path', 'Launch_H', 'Launch_V', 'DescentV',
       'Curve_Dist_yds', 'Lateral_Impact_in', 'Vertical_Impact_in']

clubs = ['Driver','3 Wood','5 Wood','4 Iron','5 Iron','6 Iron','7 Iron', '8 Iron','9 Iron', 'Pitching Wedge',  'Gap Wedge','Sand Wedge' , 'Lob Wedge']

In [109]:
#### Helper functions   ################################################################
# Convert value with error handling
def convert_value(value):
    """
    Converts any value like '40R' or '20L' to numerical form:
    - '40R' becomes -40
    - '20L' becomes 20
    Returns None for invalid entries.
    """
    try:
        value = str(value).strip()  # Ensure value is a string and remove spaces
        number, direction = value[:-1], value[-1].upper()  # Split number and direction
        number = float(number)
        return -number if direction == 'R' else number if direction == 'L' else None
    except (ValueError, AttributeError):
        return None  # Return None for invalid entries

# Convert entire column and handle missing column errors
def convert_column(df, col):
    """
    Converts all values in a column using convert_value.
    Handles missing column errors gracefully.
    """
    if col in df.columns:
        df[col] = df[col].apply(convert_value)
        #print(f"Column '{col}' successfully converted.")
    else:
        st.error(f"The column '{col}' is missing from the data.")
########################################################################################

def process_df(df):
    clubs = ['Driver','3 Wood','5 Wood','4 Iron','5 Iron','6 Iron','7 Iron', '8 Iron','9 Iron', 'Pitching Wedge',  'Gap Wedge','Sand Wedge' , 'Lob Wedge']
    # Cleaning the column names
    # Clean column names to ensure consistency
    df.columns = df.columns.str.replace(r'[^\w\s]', '', regex=True).str.replace('\xa0', ' ').str.strip().str.replace(' ', '_')

    # Convert Time to strings for Session variable
    df['Session'] = df['Time'].dt.strftime('%Y %b %d %I:%M %p')  # Or any simpler string representation

    # Convert all the columns that have L & R in the data        
    lrcols = ['Swing_H','Spin_Axis','Lateral_yds','FTP','FTT','Club_Path','Launch_H']
    for col in lrcols:
        convert_column(df,col)

    # 1. Parse & sort
    df['Session_dt'] = pd.to_datetime(df['Session'], format='%Y %b %d %I:%M %p')
    df = df.sort_values('Session_dt').copy()
    # 2. Convert Session to ordered categorical
    sorted_sessions = df['Session'].unique()
    df['Session'] = pd.Categorical(
        df['Session'],
        categories=sorted_sessions,
        ordered=True             )
    # 3. Pivot with Sessions
    df_pivot = df.pivot_table(
        index='Club',
        columns=['Golfer', 'Session'],
        values=num_columns, 
        aggfunc='mean')
    # 4. (Optional) Reindex columns to ensure Session is in ascending chronological order
    df_sessions = df_pivot.reindex(columns=sorted_sessions, level='Session')

    df_golfer = df.pivot_table(
        index='Club',
        columns=['Golfer'],
        values=num_columns, 
        aggfunc='mean')
    df_golfer = df_golfer.reindex(clubs)
    df_sessions = df_sessions.reindex(clubs)
    
    return df,df_sessions,df_golfer

In [110]:
df, df_sessions, df_golfer = process_df(df)       # #  Club in rows, Golfer, Session in Columns as MultiIndex  (all metrics)

C:\Users\dmomb\AppData\Local\Temp\ipykernel_44396\2437700188.py:55: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



In [111]:
# Calculate counts grouped by Golfer and Club
shot_counts = df.groupby(['Golfer', 'Club'])['Carry_yds'].count().reset_index()
shot_counts.rename(columns={'Carry_yds': 'Shot_Count'}, inplace=True)
# Pivot table for display
pivot_counts = shot_counts.pivot(index='Club', columns='Golfer', values='Shot_Count')
pivot_counts

Golfer,Alex,Dave,Katy
Club,,,
4 Iron,19.0,15.0,NaN
5 Iron,26.0,53.0,NaN
5 Wood,12.0,NaN,NaN
6 Iron,21.0,46.0,NaN
7 Iron,106.0,154.0,5.0
8 Iron,96.0,53.0,NaN
9 Iron,31.0,66.0,6.0
Driver,9.0,NaN,NaN
Gap Wedge,10.0,NaN,NaN


In [112]:
import plotly.io as pio

# Force the default template to "plotly"
pio.templates.default = "plotly"


# --- 2) Melt the DataFrame from wide to long ---
df_melt = df_dave.reset_index().melt(
    id_vars='Club',             # 'Club' stays as its own column
    var_name='Session',         # The old column names (session strings) become this column
    value_name='Metric'         # The cell values become 'Metric'
)

# --- 3) Parse Session to a datetime so we can plot chronologically
df_melt['Session_dt'] = pd.to_datetime(
    df_melt['Session'],
    format='%Y %b %d %I:%M %p',
    errors='coerce'
)

# --- 4) Create the line plot using Plotly Express
fig = px.line(
    data_frame=df_melt,
    x='Session_dt',   # time on the x-axis
    y='Metric',       # the numeric metric on the y-axis
    color='Club',     # one line per club
    markers=True      # optional: show markers for data points
)
# Force a known set of discrete colors
color_discrete_sequence=px.colors.qualitative.Plotly

# --- 5) Customize as needed ---
fig.update_layout(
    xaxis_title='Session',
    yaxis_title='Metric',
    title='Metric by Club over Sessions'
)


fig.update_traces(connectgaps=True)
fig.show()

In [113]:
# Function to remove outliers based on IQR for one column
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    return data[data[column] >= lower_bound]


In [114]:
# Remove outliers for 'Carry_yds' column
df_stats = remove_outliers(df, 'Carry_yds')

In [115]:
df_stats

,Mombo_ShotID,Club,Time,Golfer,Shot,Video,Ball_mph,Club_mph,Smash_Factor,Carry_yds,...,Unnamed_37,Unnamed_38,Unnamed_39,Unnamed_40,Comment,User1,User2,Exclude,Session,Session_dt
0,1,Pitching Wedge,2024-10-31 20:23:00,Dave,1,NaN,93.6,76.5,1.22,120.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
15,16,Pitching Wedge,2024-10-31 20:23:00,Dave,16,NaN,89.0,67.8,1.31,97.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
14,15,Pitching Wedge,2024-10-31 20:23:00,Dave,15,NaN,87.9,72.8,1.21,105.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
12,13,Pitching Wedge,2024-10-31 20:23:00,Dave,13,NaN,92.3,77.8,1.19,117.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
11,12,Pitching Wedge,2024-10-31 20:23:00,Dave,12,NaN,81.7,68.3,1.19,98.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,691,Pitching Wedge,2024-12-23 16:01:00,Alex,2,NaN,92.1,78.9,1.17,121.1,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
691,692,Pitching Wedge,2024-12-23 16:01:00,Alex,3,NaN,96.2,77.3,1.24,127.7,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
693,694,Pitching Wedge,2024-12-23 16:01:00,Alex,5,NaN,98.6,78.2,1.26,130.2,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
674,675,8 Iron,2024-12-23 16:01:00,Alex,40,NaN,118.1,86.9,1.36,172.0,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00


In [116]:
metric = 'Carry_yds'
golfer = 'Alex'
session = '2024-12-23 16:01:00'

def get_metric_data(df, metric, golfer, session):

# Filter rows by golfer and session
    filtered_df = df[(df['Golfer'] == golfer) & (df['Session'] == session)]
    
    # Return the values from the specified metric column
    if metric in filtered_df.columns:
        return filtered_df[metric]
    else:
        raise ValueError(f"Metric '{metric}' not found in DataFrame columns.")





In [117]:
result_series = get_metric_data(df, 'Carry_yds', 'Dave', '2024 Nov 10 02:56 PM')

In [118]:
result_series

221    108.9
222    112.1
223    100.9
224    105.3
225     97.9
226     65.7
220    127.5
219    124.0
218    130.6
217    142.3
204     82.2
205    112.3
206    120.2
207    108.6
208    163.6
209    121.6
210     46.1
211    113.6
212    126.8
213    100.5
214    112.2
215    125.6
216    121.5
Name: Carry_yds, dtype: float64

In [119]:
Q1 = result_series.quantile(0.25)
Q1

np.float64(103.1)

In [120]:
Q1

np.float64(103.1)

In [121]:
def find_outliers_iqr(series):
    """
    Returns a boolean mask indicating which values are outliers
    according to the 1.5 * IQR rule.
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outlier_mask = (series < lower_bound) | (series > upper_bound)
    return outlier_mask

# Example usage
outlier_mask = find_outliers_iqr(result_series)
print("Outlier values:")
print(result_series[outlier_mask])

Outlier values:
226     65.7
208    163.6
210     46.1
Name: Carry_yds, dtype: float64


In [122]:
def remove_outliers_iqr(group, cols):
    """
    Given a DataFrame 'group' (already filtered by a particular
    Golfer, Session, Club), remove outliers in each column in 'cols'
    using the 1.5*IQR rule.
    Returns the group without outliers.
    """
    for col in cols:
        if col not in group.columns:
            # skip if the col doesn't exist in group
            continue

        # Only consider valid (non-NaN) data
        col_data = group[col].dropna()
        if col_data.empty:
            # no data for this group in this column
            continue

        Q1 = col_data.quantile(0.25)
        Q3 = col_data.quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Create a boolean mask for outliers
        outlier_mask = (group[col] < lower_bound) | (group[col] > upper_bound)

        # Remove outliers by setting them to NaN or dropping them
        # Option A: If you want to *drop* rows with outliers:
        group = group[~outlier_mask]

        # Option B (Alternative):
        # If you want to keep the row but set the value to NaN:
        # group.loc[outlier_mask, col] = float('NaN')

    return group

# 2. Apply the function groupwise
df_no_outliers = (
    df
    .groupby(['Golfer', 'Session', 'Club'], group_keys=False,observed=False)
    .apply(lambda grp: remove_outliers_iqr(grp, num_columns))
)

C:\Users\dmomb\AppData\Local\Temp\ipykernel_44396\1965233637.py:43: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [123]:
df_no_outliers['Golfer'].unique()


array(['Alex', 'Dave', 'Katy'], dtype=object)

In [124]:
df_no_outliers

,Mombo_ShotID,Club,Time,Golfer,Shot,Video,Ball_mph,Club_mph,Smash_Factor,Carry_yds,...,Unnamed_37,Unnamed_38,Unnamed_39,Unnamed_40,Comment,User1,User2,Exclude,Session,Session_dt
56,57,7 Iron,2024-11-03 20:05:00,Alex,19,NaN,123.4,85.2,1.45,180.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
68,69,7 Iron,2024-11-03 20:05:00,Alex,12,NaN,101.5,85.0,1.19,136.7,...,NaN,NaN,NaN,NaN,Alex?,NaN,NaN,x,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
65,66,7 Iron,2024-11-03 20:05:00,Alex,9,NaN,96.1,85.3,1.13,116.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
66,67,7 Iron,2024-11-03 20:05:00,Alex,10,NaN,117.4,85.9,1.37,167.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
55,56,7 Iron,2024-11-03 20:05:00,Alex,18,NaN,107.1,85.4,1.25,147.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,62,7 Iron,2024-11-03 20:05:00,Katy,5,NaN,91.4,78.9,1.16,120.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
57,58,7 Iron,2024-11-03 20:05:00,Katy,1,NaN,54.1,44.7,1.21,42.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 03 08:05 PM,2024-11-03 20:05:00
201,202,9 Iron,2024-11-10 14:56:00,Katy,4,NaN,63.8,54.9,1.16,67.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 10 02:56 PM,2024-11-10 14:56:00
202,203,9 Iron,2024-11-10 14:56:00,Katy,5,NaN,61.6,53.6,1.15,61.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Nov 10 02:56 PM,2024-11-10 14:56:00


In [125]:
df

,Mombo_ShotID,Club,Time,Golfer,Shot,Video,Ball_mph,Club_mph,Smash_Factor,Carry_yds,...,Unnamed_37,Unnamed_38,Unnamed_39,Unnamed_40,Comment,User1,User2,Exclude,Session,Session_dt
0,1,Pitching Wedge,2024-10-31 20:23:00,Dave,1,NaN,93.6,76.5,1.22,120.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
15,16,Pitching Wedge,2024-10-31 20:23:00,Dave,16,NaN,89.0,67.8,1.31,97.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
14,15,Pitching Wedge,2024-10-31 20:23:00,Dave,15,NaN,87.9,72.8,1.21,105.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
12,13,Pitching Wedge,2024-10-31 20:23:00,Dave,13,NaN,92.3,77.8,1.19,117.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
11,12,Pitching Wedge,2024-10-31 20:23:00,Dave,12,NaN,81.7,68.3,1.19,98.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024 Oct 31 08:23 PM,2024-10-31 20:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,691,Pitching Wedge,2024-12-23 16:01:00,Alex,2,NaN,92.1,78.9,1.17,121.1,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
691,692,Pitching Wedge,2024-12-23 16:01:00,Alex,3,NaN,96.2,77.3,1.24,127.7,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
693,694,Pitching Wedge,2024-12-23 16:01:00,Alex,5,NaN,98.6,78.2,1.26,130.2,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
674,675,8 Iron,2024-12-23 16:01:00,Alex,40,NaN,118.1,86.9,1.36,172.0,...,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,2024 Dec 23 04:01 PM,2024-12-23 16:01:00
